## Capture images

In [ ]:
import time
import cv2
import datetime as dt
from notify_run import Notify
notify = Notify()

In [ ]:
def get_properties(cap):
    properties = {
        10:'CV_CAP_PROP_BRIGHTNESS',
        11:'CV_CAP_PROP_CONTRAST',
        12:'CV_CAP_PROP_SATURATION',
        13:'CV_CAP_PROP_HUE',
        14:'CV_CAP_PROP_GAIN',
        15:'CV_CAP_PROP_EXPOSURE'
    }
    
    return [cap.get(key) for key,value in properties.items()]

In [ ]:
starttime = time.time()
notification_sent = False
while True:
    try:
        cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_EXPOSURE, 7)
        timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        ret,frame = cap.read()
        props = get_properties(cap)
        cv2.imwrite(f'images/capture_{timestamp}.png',frame)
        cap.release()
        cv2.destroyAllWindows()
        
        with open("log.txt", "a+") as f:
            f.write(f'{timestamp}: {props}\n')
        
    except:
        if not notification_sent:
            notify.send('Timelapse has run into an error')
            notification_sent=True
        
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))